# Create Bronze Layer Shortcuts

This notebook guides you through creating the Bronze layer shortcuts.

## Shortcut Locations

| Source | Location | Count |
|--------|----------|-------|
| ADLS Gen2 (parquet) | **Files/** | 24 shortcuts |
| Eventhouse (streaming) | **Tables/cusn/** | 18 shortcuts |

**Note:** ADLS parquet shortcuts go in Files, Eventhouse shortcuts go in Tables.

In [ ]:
# Configuration
BRONZE_SCHEMA = "cusn"

# ADLS Gen2 settings
ADLS_ACCOUNT = "stdretail"
ADLS_CONTAINER = "supermarket"

# Eventhouse settings
EVENTHOUSE_DATABASE = "retail_eventhouse"

print(f"Bronze Schema: {BRONZE_SCHEMA}")
print(f"ADLS: https://{ADLS_ACCOUNT}.dfs.core.windows.net/{ADLS_CONTAINER}")
print(f"Eventhouse: {EVENTHOUSE_DATABASE}")

In [ ]:
# Create Bronze schema for Eventhouse shortcuts
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {BRONZE_SCHEMA}")
print(f"✓ Schema '{BRONZE_SCHEMA}' created")

## Step 1: Create ADLS Shortcuts in Files

In Fabric Portal:
1. Open your Lakehouse
2. Right-click **Files** → **New shortcut** → **Azure Data Lake Storage Gen2**
3. Create a shortcut for each folder listed below

These shortcuts point to parquet folders containing dimension and fact data.

In [ ]:
# Dimension folders (6)
dimension_tables = [
    "dim_geographies", "dim_stores", "dim_distribution_centers",
    "dim_trucks", "dim_customers", "dim_products"
]

# Fact folders (18)
fact_tables = [
    "fact_receipts", "fact_receipt_lines", "fact_store_inventory_txn",
    "fact_dc_inventory_txn", "fact_truck_moves", "fact_truck_inventory",
    "fact_foot_traffic", "fact_ble_pings", "fact_customer_zone_changes",
    "fact_marketing", "fact_online_order_headers", "fact_online_order_lines",
    "fact_payments", "fact_store_ops", "fact_stockouts",
    "fact_promotions", "fact_promo_lines", "fact_reorders"
]

all_parquet_tables = dimension_tables + fact_tables

print(f"Create these shortcuts in Files/ ({len(all_parquet_tables)} total):")
print(f"\nADLS Connection: https://{ADLS_ACCOUNT}.dfs.core.windows.net")
print(f"Container/Path: {ADLS_CONTAINER}/[folder_name]")
print()
for i, table in enumerate(all_parquet_tables, 1):
    print(f"  {i:2d}. {table}")

## Step 2: Create Eventhouse Shortcuts in Tables

In Fabric Portal:
1. Right-click **Tables** → **New shortcut** → **Microsoft OneLake**
2. Select your Eventhouse → Select the database
3. Create a shortcut for each event table in the **cusn** schema

In [ ]:
# Event tables (18)
event_tables = [
    # Transaction Events
    "receipt_created", "receipt_line_added", "payment_processed",
    # Inventory Events
    "inventory_updated", "stockout_detected", "reorder_triggered",
    # Customer Events
    "customer_entered", "customer_zone_changed", "ble_ping_detected",
    # Operational Events
    "truck_arrived", "truck_departed", "store_opened", "store_closed",
    # Marketing Events
    "ad_impression", "promotion_applied",
    # Omnichannel Events
    "online_order_created", "online_order_picked", "online_order_shipped"
]

print(f"Create these shortcuts in Tables/{BRONZE_SCHEMA}/ ({len(event_tables)} total):")
print(f"\nEventhouse Database: {EVENTHOUSE_DATABASE}")
print()
for i, table in enumerate(event_tables, 1):
    print(f"  {i:2d}. {table}")

## Step 3: Verify Shortcuts

Run the cells below to verify shortcuts were created.

In [ ]:
# Verify Files shortcuts (ADLS parquet)
from notebookutils import mssparkutils

files = mssparkutils.fs.ls("Files/")
file_shortcuts = [f.name for f in files if f.isDir]

print(f"Files/ shortcuts: {len(file_shortcuts)}/24")
print()

# Check for missing
missing = [t for t in all_parquet_tables if t not in file_shortcuts]
if missing:
    print(f"Missing: {', '.join(missing)}")
else:
    print("✓ All ADLS shortcuts created")

In [ ]:
# Verify Tables shortcuts (Eventhouse)
bronze_tables = spark.sql(f"SHOW TABLES IN {BRONZE_SCHEMA}").collect()
table_names = [t.tableName for t in bronze_tables]

print(f"Tables/{BRONZE_SCHEMA}/ shortcuts: {len(bronze_tables)}/18")
print()

# Check for missing
missing = [t for t in event_tables if t not in table_names]
if missing:
    print(f"Missing: {', '.join(missing)}")
else:
    print("✓ All Eventhouse shortcuts created")

## Step 4: Test Data Access

In [ ]:
# Test reading from shortcuts
print("Testing data access...\n")

# Test Files shortcut (parquet)
try:
    df = spark.read.parquet("Files/dim_stores")
    print(f"✓ Files/dim_stores: {df.count()} rows")
except Exception as e:
    print(f"✗ Files/dim_stores: {e}")

# Test Tables shortcut (Eventhouse)
try:
    df = spark.table(f"{BRONZE_SCHEMA}.receipt_created")
    print(f"✓ {BRONZE_SCHEMA}.receipt_created: {df.count()} rows")
except Exception as e:
    print(f"✗ {BRONZE_SCHEMA}.receipt_created: {e}")

print("\nBronze layer ready!")